In [2]:
import pygame, os, sys
from pygame.locals import *
import random

pygame.init()
reloj = pygame.time.Clock()
WIDTH, HEIGHT = 640, 480
pantalla = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Serpiente')
font_path = pygame.font.match_font('Arial')
fuente = pygame.font.Font(font_path, 32)
fuente_pequena = pygame.font.Font(font_path, 20)  # Fuente más pequeña

# Clase Ranking para manejar el Top 5
class Ranking:
    def __init__(self, archivo='ranking.txt'):
        self.archivo = archivo

    def leer(self):
        """
        Lee el archivo ranking.txt y devuelve el Top 5 como una lista de tuplas (nombre, frutas).
        """
        try:
            with open(self.archivo, 'r') as archivo:
                ranking = [linea.strip().split(',') for linea in archivo.readlines()]
                ranking = [(nombre, int(frutas)) for nombre, frutas in ranking]
                ranking.sort(key=lambda x: x[1], reverse=True)  # Ordenar de mayor a menor
                return ranking[:5]  # Devolver solo el Top 5
        except FileNotFoundError:
            return []  # Si no existe el archivo, devolver una lista vacía

    def guardar(self, nombre, frutas):
        """
        Guarda un nuevo puntaje en el archivo ranking.txt y actualiza el Top 5.
        """
        ranking = self.leer()
        ranking.append((nombre, frutas))
        ranking.sort(key=lambda x: x[1], reverse=True)  # Ordenar de mayor a menor
        ranking = ranking[:5]  # Mantener solo el Top 5
        with open(self.archivo, 'w') as archivo:
            for nombre, frutas in ranking:
                archivo.write(f'{nombre},{frutas}\n')

# Clase DatosJuego
class DatosJuego:
    def __init__(self):
        self.vidas = 3
        self.estas_muerto = False
        self.bloques = []
        self.tick = 300
        self.velocidad = 250
        self.nivel = 1
        self.cuenta_frutas = 0
        self.segmentos = 1
        self.frame = 0
        fx = random.randint(1, 38)
        fy = random.randint(1, 28)
        self.fruta = Posicion(fx, fy)
        self.bloques.append(Posicion(20, 15))
        self.bloques.append(Posicion(19, 15))
        self.direccion = 0
        self.total_frutas = 0  # Contador total de frutas

# Clase SerpienteCompetidora
class Posicion:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Función para cargar imágenes
def cargar_imagenes():
    pared = pygame.image.load('pared.png')
    fruta = pygame.image.load('fruta.png')
    serpiente = pygame.image.load('serpiente.png')

    # Crear la serpiente roja aplicando un filtro de color
    serpiente_roja = serpiente.copy()
    serpiente_roja.fill((255, 0, 0), special_flags=pygame.BLEND_MULT)

    return {'pared': pared, 'fruta': fruta, 'serpiente': serpiente, 'serpiente_roja': serpiente_roja}

# Función para cargar el mapa
def cargar_mapa(ruta):
    archivo = open(ruta, 'r')
    contenido = archivo.readlines()
    archivo.close()
    return contenido

# Función para cargar el mapa del nivel 4
def cargar_mapa_nivel4():
    return cargar_mapa('mapa_nivel4.txt')

# Función para dibujar la pantalla de inicio
def dibujar_pantalla_inicio(pantalla, ranking):
    pantalla.fill((0, 0, 0))
    texto1 = fuente.render('Bienvenido a Serpiente', True, (255, 255, 255))
    texto2 = fuente.render('Presiona ESPACIO para jugar', True, (255, 255, 255))
    cx = pantalla.get_width() / 2
    cy = pantalla.get_height() / 2
    texto1_area = texto1.get_rect(centerx=cx, top=cy - 48)
    texto2_area = texto2.get_rect(centerx=cx, top=cy)
    pantalla.blit(texto1, texto1_area)
    pantalla.blit(texto2, texto2_area)

    # Mostrar el Top 5
    top5 = ranking.leer()
    if top5:
        texto_ranking = fuente.render('Top 5 Jugadores:', True, (255, 255, 255))
        pantalla.blit(texto_ranking, (50, 50))
        for i, (nombre, frutas) in enumerate(top5):
            texto = fuente_pequena.render(f'{i + 1}. {nombre}: {frutas} frutas', True, (255, 255, 255))
            pantalla.blit(texto, (50, 80 + i * 30))

# Función para pedir el nombre del jugador
def pedir_nombre(pantalla):
    nombre = ""
    esperando = True
    while esperando:
        pantalla.fill((0, 0, 0))
        texto = fuente.render('Ingresa tu nombre:', True, (255, 255, 255))
        pantalla.blit(texto, (50, 50))
        texto_nombre = fuente.render(nombre, True, (255, 255, 255))
        pantalla.blit(texto_nombre, (50, 100))
        pygame.display.flip()

        for evento in pygame.event.get():
            if evento.type == QUIT:
                pygame.quit()
                sys.exit()
            if evento.type == KEYDOWN:
                if evento.key == K_RETURN:  # Enter
                    esperando = False
                elif evento.key == K_BACKSPACE:  # Borrar
                    nombre = nombre[:-1]
                else:
                    nombre += evento.unicode
    return nombre

# Función para dibujar el texto de "Juego Terminado"
def dibujar_texto_juego_terminado(pantalla):
    texto1 = fuente.render('Juego Terminado', True, (255, 255, 255))
    texto2 = fuente.render('Espacio para jugar o cierra la ventana', True, (255, 255, 255))
    cx = pantalla.get_width() / 2
    cy = pantalla.get_height() / 2
    texto1_area = texto1.get_rect(centerx=cx, top=cy - 48)
    texto2_area = texto2.get_rect(centerx=cx, top=cy)
    pantalla.blit(texto1, texto1_area)
    pantalla.blit(texto2, texto2_area)

# Función para dibujar las paredes
def dibujar_paredes(pantalla, imagen, mapa):
    fila = 0
    for linea in mapa:
        col = 0
        for letra in linea:
            if letra == '1':
                imagen_area = imagen.get_rect()
                imagen_area.top = fila * 16
                imagen_area.left = col * 16
                pantalla.blit(imagen, imagen_area)
            col += 1
        fila += 1

# Función para dibujar los datos del juego
def dibujar_datos(pantalla, datos):
    texto = f'Vidas = {datos.vidas}, Nivel = {datos.nivel}'
    texto = fuente.render(texto, True, (255, 255, 255))
    texto_area = texto.get_rect(centerx=pantalla.get_width() / 2, top=32)
    pantalla.blit(texto, texto_area)

# Función para dibujar la serpiente
def dibujar_serpiente(pantalla, imagen, datos):
    inicio = True
    for bloque in datos.bloques:
        dest = (bloque.x * 16, bloque.y * 16, 16, 16)
        if inicio:
            inicio = False
            fuente = (((datos.direccion * 2) + (datos.frame)) * 16, 0, 16, 16)
        else:
            fuente = (8 * 16, 0, 16, 16)
        pantalla.blit(imagen, dest, fuente)

# Función para dibujar la serpiente roja
def dibujar_serpiente_roja(pantalla, imagen, serpiente_roja):
    for bloque in serpiente_roja.bloques:
        dest = (bloque.x * 16, bloque.y * 16, 16, 16)
        pantalla.blit(imagen, dest)

# Función para dibujar el conteo de frutas
def dibujar_conteo_frutas(pantalla, imagen, datos):
    for i in range(datos.total_frutas):
        dest = (10 + i * 20, 10, 16, 16)
        pantalla.blit(imagen, dest)
    texto = fuente_pequena.render(f'Frutas comidas: {datos.total_frutas}', True, (255, 255, 255))
    pantalla.blit(texto, (10, 40))

# Función para actualizar el juego
def actualizar_juego(datos, tiempo):
    datos.tick -= tiempo
    cabeza = datos.bloques[0]
    if datos.tick < 0:
        datos.tick += datos.velocidad
        datos.frame += 1
        datos.frame %= 2
        if datos.direccion == 0:
            mov = (1, 0)
        elif datos.direccion == 1:
            mov = (-1, 0)
        elif datos.direccion == 2:
            mov = (0, -1)
        else:
            mov = (0, 1)
        nueva_pos = Posicion(cabeza.x + mov[0], cabeza.y + mov[1])
        for bloque in datos.bloques:
            temp = Posicion(bloque.x, bloque.y)
            bloque.x = nueva_pos.x
            bloque.y = nueva_pos.y
            nueva_pos = Posicion(temp.x, temp.y)

    teclas = pygame.key.get_pressed()
    if teclas[K_RIGHT] and datos.direccion != 1:
        datos.direccion = 0
    elif teclas[K_LEFT] and datos.direccion != 0:
        datos.direccion = 1
    elif teclas[K_UP] and datos.direccion != 3:
        datos.direccion = 2
    elif teclas[K_DOWN] and datos.direccion != 2:
        datos.direccion = 3

    if (cabeza.x == datos.fruta.x and cabeza.y == datos.fruta.y):
        ultimo_indice = len(datos.bloques) - 1
        for i in range(datos.segmentos):
            nueva_cola = Posicion(datos.bloques[ultimo_indice].x, datos.bloques[ultimo_indice].y)
            datos.bloques.append(nueva_cola)
        fx = random.randint(1, 38)
        fy = random.randint(1, 28)
        datos.fruta = Posicion(fx, fy)
        datos.cuenta_frutas += 1
        datos.total_frutas += 1  # Incrementar el contador total de frutas
        if datos.cuenta_frutas == 3:
    datos.cuenta_frutas = 0
    datos.velocidad -= 25
    datos.nivel += 1
    datos.segmentos *= 2
    if datos.segmentos > 64:
        datos.segmentos = 64
    if datos.velocidad < 100:
        datos.velocidad = 100

    # Reaparecer la serpiente roja en cada nuevo nivel
    if datos.nivel >= 5:
        serpiente_roja = SerpienteCompetidora()
        serpiente_roja.activa = True

# Función para verificar si la cabeza golpea una pared
def cabeza_golpea_pared(mapa, datos):
    fila = 0
    for linea in mapa:
        col = 0
        for letra in linea:
            if letra == '1':
                if datos.bloques[0].x == col and datos.bloques[0].y == fila:
                    return True
            col += 1
        fila += 1
    return False

# Función para verificar si la cabeza golpea el cuerpo
def cabeza_golpea_cuerpo(datos):
    cabeza = datos.bloques[0]
    for i in range(1, len(datos.bloques)):
        bloque = datos.bloques[i]
        if cabeza.x == bloque.x and cabeza.y == bloque.y:
            return True
    return False

# Función para perder una vida
def perder_vida(datos):
    datos.vidas -= 1
    datos.bloques[:] = []
    datos.direccion = 0
    datos.bloques.append(Posicion(20, 15))
    datos.bloques.append(Posicion(19, 15))
    global serpiente_roja
    serpiente_roja = SerpienteCompetidora()  # Reiniciar la serpiente roja

def posicion_fruta(datos):
    fx = random.randint(1, 38)
    fy = random.randint(1, 28)
    colision = True
    while colision:
        colision = False
        for bloque in datos.bloques + serpiente_roja.bloques:  # Verificar ambas serpientes
            if bloque.x == fx and bloque.y == fy:
                fx = random.randint(1, 38)
                fy = random.randint(1, 28)
                colision = True
                break
    datos.fruta = Posicion(fx, fy)

# Inicialización del juego
imagenes = cargar_imagenes()
imagenes['fruta'].set_colorkey((255, 0, 255))
mapa = cargar_mapa('mapa.txt')
datos = DatosJuego()
ranking = Ranking()
quitar_juego = False
estas_jugando = False

serpiente_roja = SerpienteCompetidora()  # Inicializar la serpiente roja

while not quitar_juego:
    if estas_jugando:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

        fruta_area = imagenes['fruta'].get_rect()
        fruta_area.top = datos.fruta.y * 16
        fruta_area.left = datos.fruta.x * 16

        actualizar_juego(datos, reloj.get_time())

        # Cambiar el mapa al nivel 4
        if datos.nivel == 4 and mapa != cargar_mapa_nivel4():
            mapa = cargar_mapa_nivel4()

        # Activar la serpiente roja a partir del nivel 5
if datos.nivel >= 5:
    serpiente_roja.activa = True
    serpiente_roja.mover_hacia_fruta(datos.fruta, mapa, reloj.get_time())

    # Verificar colisión entre el jugador y la serpiente roja
    for bloque in serpiente_roja.bloques:
        if bloque.x == datos.bloques[0].x and bloque.y == datos.bloques[0].y:
            perder_vida(datos)
            serpiente_roja.activa = False  # Desactivar la serpiente roja

    # Verificar si la serpiente roja come la fruta
    if serpiente_roja.bloques[0].x == datos.fruta.x and serpiente_roja.bloques[0].y == datos.fruta.y:
        serpiente_roja.crecer()
        posicion_fruta(datos)

        colision = cabeza_golpea_pared(mapa, datos) or cabeza_golpea_cuerpo(datos)

        if colision:
            perder_vida(datos)
            posicion_fruta(datos)

        estas_jugando = datos.vidas > 0
        if estas_jugando:
            pantalla.fill((0, 0, 0))
            pantalla.blit(imagenes['fruta'], fruta_area)
            dibujar_paredes(pantalla, imagenes['pared'], mapa)
            dibujar_datos(pantalla, datos)
            dibujar_serpiente(pantalla, imagenes['serpiente'], datos)
            dibujar_conteo_frutas(pantalla, imagenes['fruta'], datos)

            # Dibujar la serpiente roja si está activa
            if serpiente_roja.activa:
                dibujar_serpiente_roja(pantalla, imagenes['serpiente_roja'], serpiente_roja)
        else:
            # Registrar puntaje si el jugador pierde todas las vidas
            if datos.vidas <= 0:
                top5 = ranking.leer()
                if len(top5) < 5 or datos.total_frutas > top5[-1][1]:
                    # Si el puntaje entra en el Top 5, pedir el nombre y guardarlo
                    nombre = pedir_nombre(pantalla)
                    ranking.guardar(nombre, datos.total_frutas)
                else:
                    # Si no entra en el Top 5, solo mostrar el Top 5
                    pantalla.fill((0, 0, 0))
                    texto = fuente.render('No entraste al Top 5', True, (255, 255, 255))
                    pantalla.blit(texto, (50, 50))

                # Mostrar el Top 5 al final de la partida
                dibujar_pantalla_inicio(pantalla, ranking)

    else:
        teclas = pygame.key.get_pressed()

        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()

        if teclas[K_SPACE]:
            estas_jugando = True
            datos = DatosJuego()
            serpiente_roja = SerpienteCompetidora()  # Reiniciar la serpiente roja

        dibujar_pantalla_inicio(pantalla, ranking)

    pygame.display.update()
    reloj.tick(30)

IndentationError: expected an indented block after 'if' statement on line 240 (1238764622.py, line 241)